In [1]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")
                    .set("spark.executor.cores", "3")
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ContratosReadParquet").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/22 10:40:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Sat Oct 22 10:40:13 2022


In [3]:
#spark job monitoring
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esquema = StructType() \
      .add("ctId",StringType(),True) \
      .add("fecIni",StringType(),True) \
      .add("fecFin",StringType(),True) \
      .add("dato",StringType(),True) \
      .add("saldo",IntegerType(),True)

df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fakeContracts01.parquet/")\
               .format("parquet").load('./datos/fakeContracts01.parquet/proc=CT0015')

df.printSchema()

print(df.count())
df.show()
df.explain(extended=True)
df.explain(mode='cost')


root
 |-- ctId: string (nullable = true)
 |-- fecIni: string (nullable = true)
 |-- fecFin: string (nullable = true)
 |-- dato: string (nullable = true)
 |-- saldo: integer (nullable = true)
 |-- proc: string (nullable = true)



89262
+------------+----------+----------+--------------------+-----+------+
|        ctId|    fecIni|    fecFin|                dato|saldo|  proc|
+------------+----------+----------+--------------------+-----+------+
|CT0015259018|1999-12-28|2020-03-29|Dato del contrato...| 7500|CT0015|
|CT0015259018|2020-03-29|2020-04-08|Dato del contrato...|46400|CT0015|
|CT0015259018|2020-04-08|2020-05-07|Dato del contrato...|34400|CT0015|
|CT0015259018|2020-05-07|2020-06-05|Dato del contrato...|44300|CT0015|
|CT0015259018|2020-06-05|2020-07-25|Dato del contrato...| 9500|CT0015|
|CT0015259018|2020-07-25|2020-08-18|Dato del contrato...|29700|CT0015|
|CT0015259018|2020-08-18|2020-08-18|Dato del contrato...|10200|CT0015|
|CT0015259018|2020-08-18|2020-10-01|Dato del contrato...| 2900|CT0015|
|CT0015259018|2020-10-01|2020-11-20|Dato del contrato...|12800|CT0015|
|CT0015259018|2020-11-20|2020-12-04|Dato del contrato...| 7300|CT0015|
|CT0015259018|2020-12-04|2020-12-31|Dato del contrato...| 6400|CT0015|


In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Final: Sat Oct 22 10:40:19 2022
Hecho en 5.672 segundo(s)


In [5]:
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

df.groupBy('ctId').count().sort(desc('count')).show(200)

print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Inicio: Sat Oct 22 10:40:19 2022


+------------+-----+
|        ctId|count|
+------------+-----+
|CT0015315467|   16|
|CT0015340728|   16|
|CT0015315593|   16|
|CT0015274600|   16|
|CT0015401039|   16|
|CT0015352518|   16|
|CT0015288207|   16|
|CT0015293055|   16|
|CT0015302028|   16|
|CT0015268047|   16|
|CT0015266338|   16|
|CT0015312148|   16|
|CT0015279613|   16|
|CT0015386678|   16|
|CT0015322454|   16|
|CT0015325304|   16|
|CT0015331087|   16|
|CT0015308196|   16|
|CT0015349951|   16|
|CT0015331711|   16|
|CT0015390414|   16|
|CT0015277004|   16|
|CT0015297577|   16|
|CT0015271822|   16|
|CT0015290131|   16|
|CT0015341614|   16|
|CT0015385792|   16|
|CT0015326690|   16|
|CT0015371294|   16|
|CT0015276768|   16|
|CT0015340525|   16|
|CT0015289273|   16|
|CT0015269061|   16|
|CT0015343244|   16|
|CT0015366952|   16|
|CT0015371093|   16|
|CT0015369569|   16|
|CT0015301970|   16|
|CT0015314505|   16|
|CT0015322323|   16|
|CT0015287194|   16|
|CT0015269332|   16|
|CT0015312290|   16|
|CT0015290739|   16|
|CT0015381346

In [6]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
